<a href="https://colab.research.google.com/github/Sarvesh1814/HateXplain/blob/Sarvesh/LSTM_CustomLoss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import spacy
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from gensim.models import Word2Vec
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Reproducibility/Sample Model/HateXplain2.csv")

In [ ]:
df = df[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]
df['post_tokens'] = df['post_tokens'].apply(lambda x: eval(x))
df.head()

,post_tokens,Target_cat,Label_cat,final_rationales
0,"[i, dont, think, im, getting, my, baby, them, ...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[we, cannot, continue, calling, ourselves, fem...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[nawt, yall, niggers, ignoring, me]",African,normal,"[0, 0, 0, 0, 0]"
3,"[<user>, i, am, bit, confused, coz, chinese, p...",Asian,hatespeech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[this, bitch, in, whataburger, eating, a, burg...",Women,hatespeech,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
le = LabelEncoder()

# fit the encoder to your string labels
le.fit(df['Label_cat'])

# transform the string labels to integers
df['Label_cat'] = le.transform(df['Label_cat'])

In [ ]:
df

,post_tokens,Target_cat,Label_cat,final_rationales
0,"[i, dont, think, im, getting, my, baby, them, ...",None,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[we, cannot, continue, calling, ourselves, fem...",None,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[nawt, yall, niggers, ignoring, me]",African,1,"[0, 0, 0, 0, 0]"
3,"[<user>, i, am, bit, confused, coz, chinese, p...",Asian,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[this, bitch, in, whataburger, eating, a, burg...",Women,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
20143,"[if, ur, still, on, twitter, tell, carlton, i,...",None,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ..."
20144,"[when, i, first, got, on, here, and, said, i, ...",African,2,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, ..."
20145,"[was, macht, der, moslem, wenn, der, zion, geg...",Other,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
20146,"[it, is, awful, look, at, world, demographics,...",Asian,0,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, ..."


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate, Lambda
from tensorflow.keras.models import Model

# Convert tokenized tweets to sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['post_tokens'])
sequences = tokenizer.texts_to_sequences(df['post_tokens'])

# Pad sequences to a fixed length
max_len = 100
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

# Define input shapes
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
input_tweet = Input(shape=(max_len,), name='input_tweet')

# Define embedding layer with random initialization
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='embedding')(input_tweet)

# Define LSTM layer
lstm = LSTM(128, return_sequences=True)(embedding)
lstm = LSTM(128,activation='relu')(lstm)
attention_out = Attention()([lstm, lstm])

# Define dense layers for tweet classification
x = Dense(64, activation='relu')(attention_out)
x = Dense(32, activation='relu')(x)
output = Dense(3, activation='softmax', name='output')(x)

# Define model inputs and outputs
model = Model(inputs=input_tweet, outputs=output)

# Define function to calculate mask
def get_mask(y_pred):
    return tf.argmax(y_pred, axis=1)

# Define custom loss function to include mask
def custom_loss(y_true, y_pred):
    mask = tf.cast(get_mask(y_true), tf.int32)
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    mask_loss = tf.gather(loss, mask)
    return tf.reduce_mean(mask_loss)

# Compile model with custom loss function and Adam optimizer
model.compile(loss=custom_loss, optimizer='adam', metrics=['accuracy'])

# Define training and validation data
output_cat = tf.keras.utils.to_categorical(df['Label_cat'])
split = int(len(sequences) * 0.8)
input_train = sequences[:split]
output_train = output_cat[:split]
input_val = sequences[split:]
output_val = output_cat[split:]

# Train model with input and output data
history=model.fit(input_train, output_train, validation_data=(input_val, output_val), epochs=10)



Epoch 1/10
504/504 [==============================] - 149s 274ms/step - loss: 1.0833 - accuracy: 0.4195 - val_loss: 1.1520 - val_accuracy: 0.2268
Epoch 2/10
504/504 [==============================] - 109s 215ms/step - loss: 1.0830 - accuracy: 0.4281 - val_loss: 1.2137 - val_accuracy: 0.2268
Epoch 3/10
504/504 [==============================] - 110s 219ms/step - loss: 1.0770 - accuracy: 0.4281 - val_loss: 1.1740 - val_accuracy: 0.2268
Epoch 4/10
504/504 [==============================] - 107s 212ms/step - loss: 1.0907 - accuracy: 0.4281 - val_loss: 1.1329 - val_accuracy: 0.2268
Epoch 5/10
504/504 [==============================] - 105s 208ms/step - loss: 1.0812 - accuracy: 0.4281 - val_loss: 1.1602 - val_accuracy: 0.2268
Epoch 6/10
504/504 [==============================] - 104s 206ms/step - loss: 1.0778 - accuracy: 0.4281 - val_loss: 1.1862 - val_accuracy: 0.2268
Epoch 7/10
504/504 [==============================] - 104s 206ms/step - loss: 1.0811 - accuracy: 0.4281 - val_loss: 1.1619 -